This notebook is an expanded version of the analysis done here: [Price
Elasticity with
R](http://www.salemmarafi.com/code/price-elasticity-with-r)

Also you can see this page as well for a similar version: [Pricing
Analysis
Example](https://rstudio-pubs-static.s3.amazonaws.com/185880_f9e3e3ee84314609b531524dfa3f4f1b.html)

First make sure RCurl is installed (if needed) and loaded

In [ ]:
if (!require("RCurl")) install.packages("RCurl")
library("RCurl")

We use the getURL function from the RCurl package to load the data from
the website. It is returned to us as a text string.

In [ ]:
url<- "http://www.salemmarafi.com/wp-content/uploads/2014/06/supermarket.csv"
textString<-getURL(url)
df<-read.csv(text=textString)

Here is the structure of the data set.

In [ ]:
str(df)

First we have 30 observations for demand and price for eggs. Then we
have 30 observations of the ad type that was run, where 0 means featured
the cookies poster and 1 means featured the eggs poster. Finally we have
data on the price of cookies.

Here is the data:

In [ ]:
df

Let’s start with a summary of the data:

In [ ]:
summary(df)

The mean demand is 30 and the mean price is about \$4.4263333

Lets look at the distribution of demand.

In [ ]:
par(mfrow=c(1,2))
boxplot(df$Sales,main="Boxplot for Egg Sales", xlab="Egg Sales")
hist(df$Sales,main='Histogram for Egg Sales',xlab='Egg Sales',prob=T)

There are no outliers or other unusual data to speak of here.

Lets look at the correlations between the variables.

In [ ]:
cor(df)
pairs(df, pch=20)

It looks like there is a strong correlation between sales and own price.
We expected that of course.

There is a weaker correlation between egg sales and price of cookies.

Let’s model using all the predictors:

In [ ]:
model <- lm(Sales~Price.Eggs+Ad.Type+Price.Cookies, df)
summary(model)
b<-round(unname(coefficients(model)),2)
b

This shows the coefficients for the various predictors.

Price of eggs, the ad type and price of cookies are all significant with
p-value less than .05.

The multiple regression equation is:

$$
    Q = 137.37  
        -16.12(P_{eggs}) + 
    4.15(AdType)  
    -8.71(P_{cookies}) 
$$

Next we show the calculation of the price elasticity: For price
elasticity we compute the percent change in quantity divided by the
percent change in price.

$$
\text{Price Elasticity} = \frac{\frac{\Delta Q}{Q}}{\frac{\Delta P_{eggs}}{P_{eggs}}}
            =(\frac{\Delta Q}{\Delta P_{eggs}})(\frac{P_{eggs}}{Q})
$$

For the first part $\frac{\Delta Q}{\Delta P_{eggs}}$, this is just the
multiple regression coefficient describing the rate of change of $Q$ as
$P_{eggs}$ changes. That is actually just the coefficient from the
multiple regression equation! So that is just $b[2] = -16.12$. For
$\frac{P_{eggs}}{Q}$, we will use the mean of the price of eggs and the
mean of sales which we saw in the summary of the data above for that.

Let’s compute the values:

In [ ]:
priceEggs <- mean(df$Price.Eggs)
priceEggs
quantity <- mean(df$Sales)
quantity
priceElasticity <- b[2]* (priceEggs/quantity)
priceElasticity

The equation looks like this:

$$
\text{PriceElasticity} =
    \frac{ \frac{\Delta Q}{Q} }{ \frac{\Delta P_{eggs}}{P_{eggs}} } =
    \frac{\Delta Q}{\Delta P_{eggs}} \frac{P_{eggs}}{Q} =
    (-16.12)(\frac{4.43}{30}) =
    -2.38
$$

This suggests that a 1% decrease in price of eggs will increase egg
sales by 2.4%.

Next is the calculation of the Cross Price Elasticity: For cross-price
elasticity of eggs with cookies we compute the percent change in
quantity of eggs divided by the percent change in the price of cookies:

In [ ]:
PriceCookies <- mean(df$Price.Cookies)
Quantity <- mean(df$Sales)
CrossPriceElasticity <- b[4]*PriceCookies/Quantity
CrossPriceElasticity

The equation looks like this:

$$
\text{CrossPriceElasticity} =
    \frac{ \frac{\Delta Q}{Q} }{ \frac{\Delta P_{cookies}}{P_{cookies}} } =
    \frac{\Delta Q}{\Delta P_{cookies}} \frac{P_{cookies}}{Q} =
    (-8.71)(\frac{4.37}{30}) =
    -1.27
$$

This suggests that 1% decrease in cookies price will increase the egg
sales by 1.3%, and that cookies and eggs are compliments.

For setting the optimal price we need some values for $P_{cookies}$ and
$AdType$. Suppose the current cookies price is $P_{cookies}=4.37$ and
the $AdType = 0$ for featuring the cookies. We plug these into the
multiple regression equation:

$$
    Q = 137.37  
        -16.12(P_{eggs}) + 
    4.15(0)  
    -8.71(4.37) 
$$

So the slope and intercept for the simplified demand equation we will
use is this:

In [ ]:
slope<- b[2]
slope
intercept <- b[1] + b[4]*PriceCookies
intercept

So our new equation is just this:

$$   
Q  = 99.28  -16.12(P_{eggs})
$$

Suppose the unit cost of eggs is \$1.50. Then here is the profit
function:

In [ ]:
profit <- function(p) {
    (p-1.50)*(intercept+slope*p)
}
curve(profit, from=0, to =8, main="Profit Function", xlab="Price", ylab="Profit")

Finally we optimize the profit function:

In [ ]:
optimize(profit, lower=0, upper=8, maximum=TRUE)

The maximum profit occurs when the price is \$3.83 and the Profit is
\$87.53.

So we would advise to set the price of eggs to \$3.83.

That is where the maximum profit should occur!!